<a href="https://colab.research.google.com/github/feliciakiani/PolitikPedia/blob/main/Machine%20Learning/semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU \
  "pinecone-client[grpc]"==2.2.1 \
  pinecone-datasets=='0.5.0rc11' \
  sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 20.2 MB/s e

In [ ]:
from pinecone_datasets import load_dataset

dataset = load_dataset('quora_all-MiniLM-L6-bm25')
# we drop sparse_values as they are not needed for this example
dataset.documents.drop(['metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
# we will use 80K rows of the dataset between rows 240K -> 320K
dataset.documents.drop(dataset.documents.index[320_000:], inplace=True)
dataset.documents.drop(dataset.documents.index[:240_000], inplace=True)
dataset.head()

,id,values,sparse_values,metadata
240000,515997,"[-0.00531694, 0.06937869, -0.0092854, 0.003286...","{'indices': [845, 1657, 13677, 20780, 27058, 2...","{'text': ' Why is a ""law of sciences"" importan..."
240001,515998,"[-0.09243751, 0.065432355, -0.06946959, 0.0669...","{'indices': [2110, 6324, 9754, 13677, 15207, 2...",{'text': ' Is it possible to format a BitLocke...
240002,515999,"[-0.021924071, 0.032280188, -0.020190848, 0.07...","{'indices': [2110, 4949, 23579, 23758, 27058, ...",{'text': ' Can formatting a hard drive stress ...
240003,516000,"[-0.120020054, 0.024080949, 0.10693012, -0.018...","{'indices': [22014, 24734, 24773, 25791, 25991...",{'text': ' Are the new Samsung Galaxy J7 and J...
240004,516001,"[-0.095293395, -0.048446465, -0.017618902, -0....","{'indices': [307, 2110, 5785, 12969, 12971, 13...",{'text': ' I just watched an add for Indonesia...


In [ ]:
import pandas as pd
from pinecone_datasets import load_dataset

# Load the dataset
dataset = load_dataset('quora_all-MiniLM-L6-bm25')

# Drop sparse_values as they are not needed for this example
dataset.documents.drop(['metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)

# Keep only rows between 240K and 320K
subset_dataset = dataset.documents.iloc[240_000:320_000].copy()

# Save the dataset to a CSV file
csv_filename = 'quora_dataset.csv'
dataset.documents.to_csv(csv_filename, index=False)

# Display the first few rows of the dataset
print(dataset.head())

In [ ]:
print(len(dataset))

In [ ]:
import os
import pinecone

# get api key from app.pinecone.io
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
# find your environment next to the api key in pinecone console
PINECONE_ENV = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

In [ ]:
index_name = 'semantic-search-fast'

In [ ]:
import time

# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=len(dataset.documents.iloc[0]['values']),
        metric='cosine'
    )
    # wait a moment for the index to be fully initialized
    time.sleep(1)

# now connect to the index
index = pinecone.GRPCIndex(index_name)

In [ ]:
for batch in dataset.iter_documents(batch_size=100):
    index.upsert(batch)

NameError: ignored

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [ ]:
query = "which city has the highest population in the world?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=5, include_metadata=True)
xc

{'matches': [{'id': '210880',
              'metadata': {'text': ' Which is the most populated city in the '
                                   'world.?'},
              'score': 0.8828482,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': '197238',
              'metadata': {'text': ' What is the most populated city in the '
                                   'world?'},
              'score': 0.8788713,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': '239582',
              'metadata': {'text': ' Which is the largest city in the world?'},
              'score': 0.81348896,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': '216678',
              'metadata': {'text': ' Which is biggest city in the world?'},
              'score': 0.795525,
              'sparse_values': {'indices': [], 'values': []},
   

In [ ]:
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

0.88:  Which is the most populated city in the world.?
0.88:  What is the most populated city in the world?
0.81:  Which is the largest city in the world?
0.8:  Which is biggest city in the world?
0.79:  What's the world's largest city?


In [ ]:
query = "which metropolis has the highest number of people?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=5, include_metadata=True)
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

0.73:  Which is the most populated city in the world.?
0.72:  What is the most populated city in the world?
0.67:  What is the most isolated city in the world, with over a million metro area inhabitants?
0.64:  What is the biggest city?
0.62:  What is the city in the U.S. experiencing the fastest population growth as of 2016? Why?


In [ ]:
pinecone.delete_index(index_name)

In [ ]:
import pandas as pd
import os
!pip install pinecone

ERROR: Could not find a version that satisfies the requirement pinecone (from versions: none)
ERROR: No matching distribution found for pinecone


In [ ]:
import pandas as pd
import os
import pinecone
import time
from sentence_transformers import SentenceTransformer
import torch

# Load your CSV file into a Pandas DataFrame
csv_file_path = 'dataset_semantic.csv'  # Replace with the actual path to your CSV file
your_dataset = pd.read_csv(csv_file_path)

# Display the first few rows of the dataset
print(your_dataset.head())


   ID                                         Pertanyaan  \
0   1              Bagaimana sistem pemilu di Indonesia?   
1   2  Apa saja prinsip dasar dari sebuah partai poli...   
2   3  Siapa pendiri partai politik pertama di Indone...   
3   4    Bagaimana cara bergabung dengan partai politik?   
4   5            Apa peran fraksi dalam sistem parlemen?   

                    Pencarian  
0              Partai Politik  
1              Partai Politik  
2      Sejarah Partai Politik  
3  Partai Politik Pendaftaran  
4       Partai Politik Fraksi  


In [ ]:
# Initialize Pinecone and create/connect to the index

pinecone.init(
	api_key='627e6f1e-6e76-42e2-9535-ce19ee4c9197',
	environment='gcp-starter'
)

index_name= pinecone.Index('semantic-search')

# Check if the index_name is not in the list of indexes
existing_indexes = pinecone.list_indexes()
if index_name not in existing_indexes:
    # Encode an example question to get its vector representation
    example_question = your_dataset['Pertanyaan'].iloc[0]
    model = SentenceTransformer('all-MiniLM-L6-v2')  # No need to specify device, it will automatically use GPU if available
    example_vector = model.encode(example_question).tolist()

    # Create the index
    pinecone.create_index(
        name=index_name,
        dimension=len(example_vector),
        metric='cosine'
    )
    time.sleep(1)

    print("Index created successfully!")

# Encode and upsert documents into Pinecone index
for _, row in your_dataset.iterrows():
    question = row['Pertanyaan']
    vector = model.encode(question).tolist()

    # Upsert data into the Pinecone index
    pinecone.upsert(index_name, ids=row['ID'], vectors=[vector])

# Retrieve and print all data from the index
index_info = pinecone.info(index_name)
indexed_data = index_info["data"]
print("Indexed Data:")
for item in indexed_data:
    print(item)


ApiTypeError: ignored

In [ ]:
# Perform a semantic search query
query = "Bagaimana sistem pemilu di Indonesia?"
xq = model.encode(query).tolist()
result = index.query(xq, top_k=5, include_metadata=True)

# Display the search results
if 'matches' in result:
    for match in result['matches']:
        print(f"Score: {round(match['score'], 2)} - Pertanyaan: {match['metadata']['Pertanyaan']}")
else:
    print("No matches found.")


# New Section

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Load your dataset
df = pd.read_csv('dataset_semantic.csv')

# Encode questions using Universal Sentence Encoder
question_embeddings = embed(df['Pertanyaan'])

# Define your query
query = "Bagaimana sistem pemilu di Indonesia?,Partai Politik"

# Encode the query
query_embedding = embed([query])[0]

# Calculate cosine similarity between the query and dataset
cosine_similarities = cosine_similarity([query_embedding], question_embeddings)[0]

# Combine similarities with questions
results = list(zip(cosine_similarities, df['Pertanyaan'], df['Pencarian']))

# Sort the results by similarity in descending order
results.sort(reverse=True, key=lambda x: x[0])

# Print the results
for similarity, question, category in results:
    print(f"{similarity:.2f}: {question} ({category})")


Streaming output truncated to the last 5000 lines.
0.71: Apa saja jenis partai politik di Indonesia? (Proses Pemilu)
0.71: Apa saja jenis partai politik di Indonesia? (Peran Pemuda Politik)
0.71: Apa saja jenis partai politik di Indonesia? (Politik Koalisi)
0.71: Apa saja jenis partai politik di Indonesia? (Pengaruh Partai Politik)
0.71: Apa saja jenis partai politik di Indonesia? (Pengaruh Partai Politik)
0.71: Apa saja jenis partai politik di Indonesia? (Proses Pemilu)
0.71: Apa saja jenis partai politik di Indonesia? (Jenis Partai Politik)
0.71: Apa saja jenis partai politik di Indonesia? (Politik Koalisi)
0.71: Apa saja jenis partai politik di Indonesia? (Pengaruh Partai Politik)
0.71: Apa saja jenis partai politik di Indonesia? (Peran Pemuda Politik)
0.71: Apa saja jenis partai politik di Indonesia? (Pengaruh Partai Politik)
0.71: Apa saja jenis partai politik di Indonesia? (Politik Koalisi)
0.71: Apa saja jenis partai politik di Indonesia? (Peran Pemuda Politik)
0.71: Apa saja je

In [1]:
# Print the highest similarity score and corresponding sentence
highest_score, highest_question, highest_category = results[0]
print(f"\nHighest Similarity Score: {highest_score:.2f}")
print(f"Highest Similarity Question: {highest_question} ({highest_category})")

NameError: ignored

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Lambda
from sklearn.metrics.pairwise import cosine_similarity

# Load your dataset
df = pd.read_csv('dataset_semantic.csv')

# Define your dataset
questions = df['Pertanyaan'].values
categories = df['Pencarian'].values

# Tokenize the questions
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions)
sequences = tokenizer.texts_to_sequences(questions)

# Pad sequences for consistent input length
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences)

In [20]:
# Create the model using Sequential API
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=padded_sequences.shape[1]),
    GlobalAveragePooling1D(),
    Dense(50, activation='relu'),
    Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
])


# Compile the model with default cosine similarity metric (you can customize the optimizer and loss)
model.compile(optimizer='adam', loss='mse', metrics=['cosine_similarity'])

# Train the model
target_data = model.predict(padded_sequences)  # Use the same embeddings for target data
model.fit(padded_sequences, target_data, epochs=10, batch_size=32)

313/313 [==============================] - 1s 1ms/step
Epoch 1/10
313/313 [==============================] - 4s 10ms/step - loss: 7.3251e-06 - cosine_similarity: 0.9998
Epoch 2/10
313/313 [==============================] - 2s 6ms/step - loss: 4.6476e-06 - cosine_similarity: 0.9999
Epoch 3/10
313/313 [==============================] - 1s 4ms/step - loss: 4.1271e-06 - cosine_similarity: 0.9999
Epoch 4/10
313/313 [==============================] - 1s 4ms/step - loss: 3.0812e-06 - cosine_similarity: 0.9999
Epoch 5/10
313/313 [==============================] - 1s 4ms/step - loss: 3.0298e-06 - cosine_similarity: 0.9999
Epoch 6/10
313/313 [==============================] - 1s 4ms/step - loss: 2.9136e-06 - cosine_similarity: 0.9999
Epoch 7/10
313/313 [==============================] - 1s 4ms/step - loss: 1.8066e-06 - cosine_similarity: 1.0000
Epoch 8/10
313/313 [==============================] - 1s 4ms/step - loss: 1.9117e-06 - cosine_similarity: 1.0000
Epoch 9/10
313/313 [====================

In [ ]:
# Encode the query
query = "Apa itu politik"
query_sequence = tokenizer.texts_to_sequences([query])
query_padded = tf.keras.preprocessing.sequence.pad_sequences(query_sequence, maxlen=padded_sequences.shape[1])

# Get the embeddings for the query
query_embedding = model.predict(query_padded)

# Calculate cosine similarity between the query embedding and dataset embeddings
cosine_similarities = cosine_similarity(query_embedding, model.predict(padded_sequences))

# Combine similarities with questions
results = list(zip(cosine_similarities[0], questions, categories))

# Sort the results by similarity in descending order
results.sort(reverse=True, key=lambda x: x[0])

# Print the results
for similarity, question, category in results:
    print(f"{similarity:.2f}: {question} ({category})")


In [12]:
# Print the highest similarity score and corresponding sentence
highest_score, highest_question, highest_category = results[0]
print(f"\nHighest Similarity Score: {highest_score:.2f}")
print(f"Highest Similarity Question: {highest_question} ({highest_category})")


Highest Similarity Score: 0.97
Highest Similarity Question: Apa itu koalisi dalam politik? (Politik Koalisi)


In [21]:
# Save the entire model to a file
model.save("semantic_search_model")

# Save only the weights of the model to a file
model.save_weights("semantic_search_model_weights.h5")


In [ ]:
from google.colab import files

# Save the best model checkpoint to a file
files.download('/content/semantic_search_model')

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the weights into the model
weights_path = "/content/semantic_search_model_weights.h5"
model.load_weights(weights_path)

# Encode the query
query = "Apa itu politik"
query_sequence = tokenizer.texts_to_sequences([query])
query_padded = tf.keras.preprocessing.sequence.pad_sequences(query_sequence, maxlen=padded_sequences.shape[1])

# Get the embeddings for the query
query_embedding = model.predict(query_padded)

# Calculate cosine similarity between the query embedding and dataset embeddings
cosine_similarities = cosine_similarity(query_embedding, model.predict(padded_sequences))

# Combine similarities with questions
results = list(zip(cosine_similarities[0], questions, categories))

# Sort the results by similarity in descending order
results.sort(reverse=True, key=lambda x: x[0])

# Print the results
for similarity, question, category in results:
    print(f"{similarity:.2f}: {question} ({category})")

In [27]:
# Save the questions to a text file
with open('/content/questions.txt', 'w') as file:
    for question in questions:
        file.write(f"{question}\n")

# Download the text file
from google.colab import files
files.download('/content/questions.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>